# Continuous Time 2D Model

We model the example of short-lived temperature sensors on the plane $[0,1]^2$ in continuous time, using a combination of a queueing model and point process. Throughout the setup of the model, fix a time $T;$ for each time $t \in [0,T]$ the set of sensors $S_t$ at time $t$ is a set of locations on $\mathbb{R}^2,$ where $|S_t|$ is a random variable, and $S_t(i),$ the location of the $i$th sensor is uniformly sampled from the uniform measure on $[0,1]^2$.

Perhaps the easiest way to set up the model is via a (FIFO) queueing model, or birth-death process representing the sensors:

- We start with $L_0$ number of sensors, each sampled uniformly from the sphere. 
- Sensors are added to the queue every so often, again sampled uniformly from the sphere. 
- Independently of the creation of new sensors, sensors break down at a perhaps different rate, and leave the queue.

We can model the time between consecutive joining/ exiting of the queue by exponential distributions.


## Queueing Model Implementation
Consider a queueing model whose initial length is $L_0,$ the number of initial sensors we begin with. We assume that the queue processes sensors (physically: sensors break down) in a random time $\sim \text{Exp}(\lambda_1),$ and sensors join the queue with interarrival times distributed according to $\text{Exp}(\lambda_2),$ which are independent of the times it takes for the queue to process the input.

We can simulate this queue in the following way. For a given fixed time $t \in [0,T],$ by the memoryless property of exponential distributions, the time until a sensor leaves the queue is still $X_1 \sim \text{Exp}(\lambda_1)$ and the interarrival time is still distributed according to $X_2 \text{Exp}(\lambda_2).$ Now, the time until one of these two actions occurs is a random variable $Z = \min(X_1,X_2) \sim \text{Exp}(\lambda_1+\lambda_2)$ (which can be checked). Now the probability that a sensor joins the queue first is $\mathbb{P}(X_1 > X_2) = \frac{\lambda_1}{\lambda_1 + \lambda_2},$ and one minus that for the probability that a sensor breaks and leaves the queue. We can then model the process by waiting $\text{Exp}(\lambda_1+\lambda_2)$ amount of time, and with probability $\frac{\lambda_1}{\lambda_1 + \lambda_2},$ adding a new sensor to the queue, or otherwise removing the oldest sensor from the queue.

## Dynamic Network Analysis
The continuous time model allows us to subsample as finely as we want. We keep track of the birth/death of the sensors in an interval tree; for any $t \in [0,T]$ we query the interval tree for the set of points that are present on the sphere at that time, and then create a dynamic network via the delaunay construction, assigining node values via some observation function representing temperature, or another desired quantity, and the edge weights by distance on the sphere.

In [ ]:
# Come up with some periodic function on the [0,1] grid; sample random geometric graphs with the proper connectivitty.


# Do the same in 3D?

## add drift to the function, or some isometry...

In [1]:
from __future__ import division
import numpy as np
import sys
import os
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.spatial.distance import squareform
import importlib
import scipy.spatial as sp

## Load in sphere code
sys.path.append('../data/simple_egs/')
import PlaneExample as plane
import ContinuousTimeSensorModel as ctsm
importlib.reload(ctsm)
importlib.reload(plane)

## Load in d.n. analysis code
sys.path.append('../shared_scripts/')
import graph_fns as gf
import persistence_fns as pf
import sliding_window_fns as sw
from ripser import ripser
from persim import plot_diagrams
from sklearn import manifold

In [2]:
## Setup Sensor Lifetimes
T = 15 # Period 

obsfn = lambda t, p: plane.periodic_plane_nonstationary(t,p,T)
edge_wtsfn = lambda p: plane.get_edge_wts_rgg(p, threshold = plane.critical_rgg_scaling(p.shape[0]))

lambda1 = 50
lambda2 = lambda1

sensor_lifetimes = ctsm.get_sensor_lifetimes(2.5*T, lambda1, lambda2, manifold = 'plane')

In [3]:
## Create the Dynamic Network
dynamic_network_samples = 250
ts = np.linspace(0,2*T,dynamic_network_samples)

(node_wts,edge_wts, allpoints) = ctsm.sample_dynamic_network(sensor_lifetimes, ts, obsfn = obsfn,
                                    edge_wtsfn = edge_wtsfn, manifold = 'plane')

In [5]:
## Apply the analysis
import SphereExample as sphere
# apply phi functions, and scale the weights
phi_node_wts, phi_edge_wts = gf.weight_fn(node_wts, edge_wts, lamda=1, phi=sphere.identity_phi_fn)

# construct the filtrations / simplicial complexes according to our construction
filtration_matrix = list(map(lambda n, e: pf.get_filtration(n, e), phi_node_wts, phi_edge_wts))

# summarize these filtrations using H_0 barcodes
barcodes = list(map(pf.get_rips_complex, filtration_matrix))

In [6]:
barcodes

[array([[ 2., inf]]),
 array([[2.07226539,        inf]]),
 array([[2.28829384,        inf]]),
 array([[2.63800788,        inf]]),
 array([[3.11458683,        inf]]),
 array([[3.67836738,        inf]]),
 array([[4.46889353,        inf]]),
 array([[4.82400703,        inf]]),
 array([[4.55950546,        inf]]),
 array([[3.74634147,        inf]]),
 array([[2.36232686,        inf]]),
 array([[0.00154735, 0.00307659],
        [0.        , 0.00593376],
        [0.        , 0.00678286],
        [0.        , 0.00797789],
        [0.        , 0.00808234],
        [0.        , 0.00877602],
        [0.        , 0.00913185],
        [0.        , 0.00957138],
        [0.0087932 , 0.00971134],
        [0.        , 0.00997106],
        [0.        , 0.01024623],
        [0.        , 0.01025476],
        [0.        , 0.01055382],
        [0.        , 0.01063162],
        [0.        , 0.01078954],
        [0.        , 0.01098876],
        [0.        , 0.01100394],
        [0.        , 0.01155579],
      